## Download ECMWF xml

In [24]:
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import xarray as xr
import numpy as np

In [15]:
save_dir = Path("/home/turnerm/sync/aa_repo_data/Data/public/exploration/phl/ecmwf_hindcast")

## Get typhoon names

In [16]:
filename = "../../IBF-Typhoon-model/data/wind_data/input/typhoon_events.csv"
df_typhoons = pd.read_csv(filename)
df_typhoons.columns = ["local", "international", "year"]
for cname in ["local", "international"]:
    df_typhoons[cname] = df_typhoons[cname].str.lower()
df_typhoons = df_typhoons.sort_values(by=["year", "international"])

## Test with Durian

In [17]:
# Select a typhoon and a target date
typhoon_name = "durian"
landfall_date = datetime(2006, 11, 30, 12)

In [18]:
filename = save_dir / f"csv/{typhoon_name}_all.csv"
forecast_date = landfall_date - timedelta(days=3)

df = pd.read_csv(filename)
for cname in ["time", "forecast_time"]:
    df[cname] = pd.to_datetime(df[cname])
df = df[df["mtype"] == "ensembleforecast"]
df = df[df["forecast_time"] == forecast_date]

In [29]:
df

,mtype,product,cyc_number,ensemble,speed,pressure,time,lat,lon,lead_time,forecast_time
1185,ensembleforecast,ecmwf,25,0,13.3,1004.8,2006-11-27 12:00:00,9.8,136.8,0,2006-11-27 12:00:00
1186,ensembleforecast,ecmwf,25,0,13.6,1005.5,2006-11-28 00:00:00,10.4,134.2,12,2006-11-27 12:00:00
1187,ensembleforecast,ecmwf,25,0,13.5,1005.3,2006-11-28 12:00:00,11.3,130.6,24,2006-11-27 12:00:00
1188,ensembleforecast,ecmwf,25,0,13.6,1005.1,2006-11-29 00:00:00,12.6,128.5,36,2006-11-27 12:00:00
1189,ensembleforecast,ecmwf,25,0,21.3,1000.0,2006-11-29 12:00:00,13.0,126.0,48,2006-11-27 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...
1741,ensembleforecast,ecmwf,25,50,20.0,999.8,2006-11-30 12:00:00,12.3,122.0,72,2006-11-27 12:00:00
1742,ensembleforecast,ecmwf,25,50,28.3,992.1,2006-12-01 00:00:00,12.3,119.9,84,2006-11-27 12:00:00
1743,ensembleforecast,ecmwf,25,50,36.7,984.1,2006-12-01 12:00:00,13.0,117.9,96,2006-11-27 12:00:00
1744,ensembleforecast,ecmwf,25,50,32.7,983.0,2006-12-02 00:00:00,13.0,115.8,108,2006-11-27 12:00:00


In [76]:
track_data = []

for ensemble, group in df.groupby("ensemble"):

    time_step = (group["time"].values[1] - group["time"].values[0]).astype('timedelta64[h]')
    time_step = pd.to_timedelta(time_step).total_seconds() / 3600

    coords = dict(
         time=(["time"], group["time"]),
    )
    data_vars=dict(
        max_sustained_wind=(["time"], group['speed']),
        central_pressure=(["time"], group['pressure']),
        lat=(["time"], group["lat"]),
        lon=(["time"], group["lon"]),
        time_step=(["time"], [time_step] * len(group))
    )
    attrs=dict(
        max_sustained_wind_unit="m/s",
        central_pressure_unit="mb",
        name=typhoon_name.upper(),
        data_provider="ECMWF",
        ensemble_number=ensemble,
        is_ensemble=True,
        forecast_time=forecast_date,
        basin="W - North West Pacific"
    )

    ds = xr.Dataset(data_vars=data_vars, coords=coords, attrs=attrs).set_coords(["lat", "lon"])
    track_data.append(ds)

In [78]:
len(track_data)

51

In [69]:
time_step = (group["time"].values[1] - group["time"].values[0]).astype('timedelta64[h]')
time_step = pd.to_timedelta(time_step).total_seconds() / 3600
time_step

12.0